<h2> Most Streamed Spotify Songs Analysis </h2>
<p> In this notebook I run through some exploratory work with this dataset and then I begin to apply some more in depth analysis to produce a model that predicts the amount of streams a song recieves. As you look through the code I want you to keep in mind that this model is still being worked on in terms of performance. I suppose you can call it somewhat of a start. I enjoy applying my skills and this is, in my opinion, the best way for me to do it. 
    
<p> I tried my best to comment and explain my train of thought, but I apologize if I lose you along the way. My brain works in an interesting way at times. Cheers! </p>

<h3> Initial Packages </h3>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from scipy import stats

In [ ]:
# Attempting to load the dataset using 'ISO-8859-1' encoding
spotify_raw_data = pd.read_csv(r"C:\Users\Steph\Desktop\Python\spotify-2023.csv", encoding='ISO-8859-1')

# Displaying the first few rows of the dataset
spotify_raw_data.head()

# Assigning data to dataframe
df = pd.DataFrame(spotify_raw_data)
print(df.shape) # (953 , 24)

# Check for nulls
print("\nMissing Values:")
print(spotify_raw_data.isnull().sum()) # [in_shazam_charts],[key] contain null values

# Summary statistics 
print("\nSummary Statistics:")
data_sumstats = spotify_raw_data.describe(include='all')
data_sumstats

<h3>Data Preprocessing</h3>
<p> In these next few steps I aim to remove the nulls as well as any potential duplicates or data type errors. </p>

In [ ]:
# Looking at missing values again 
df.isnull().sum()

# Decide whether to drop data or input missing data
# In_shazam_charts empty values can be taken as 0
# Key is more difficult and best option is to 1) Find the data 2) Remove rows and the consequences that come with less data
  # Moving forward with removing rows for now
    
# in_shazam_charts
df['in_shazam_charts'] = df['in_shazam_charts'].fillna(0)

# using dropna from pandas to remove nulls
df = df.dropna()
df.isnull().sum() 

# Expecting a 95 row decrease - validating
print(df.shape)

In [ ]:
# Check for duplicates
df.drop_duplicates() 

# Checking if rows were dropped
print(df.shape)

In [ ]:
# Looking at missing values again to validate above
df.isnull().sum()

In [ ]:
# Looking at datatypes to see what needs to be changed
print(df.dtypes)

In [ ]:
# the below are expected to be numerical

df['streams'] = pd.to_numeric(df['streams'], errors = 'coerce')
df = df.dropna() # Do not want to input 0 for streams. Will be dropping.

# numbers are objects due to commas
df['in_shazam_charts'] = df['in_shazam_charts'].str.replace(',', '')
df = df.dropna() # Do not want to input 0 for streams. Will be dropping.
df['in_shazam_charts'] = df['in_shazam_charts'].astype(int)

# numbers are objects due to commas
df['in_deezer_playlists'] = df['in_deezer_playlists'].str.replace(',', '')
df = df.dropna() # Do not want to input 0 for streams. Will be dropping.
df['in_deezer_playlists'] = df['in_deezer_playlists'].astype(int)

# Looking at datatypes
print(df.dtypes)

In [ ]:
# Looking at null values
df.isnull().sum()
print(df.shape)

<h2> Categorical Variables </h2>
<p> I like to add indicator variables for my categories. I may not use them all, but they are good to add in the dataset just in case </p>

In [ ]:
# Encode Categorical Features
 # I like to use get dummies because of the naming convention

# Categorizing [mode]
df = pd.get_dummies(df, columns=["mode"], prefix = "mode")

# Categorizing [key]
df = pd.get_dummies(df, columns=["key"], prefix = "key")

# Converting from bool to int
for col in df.columns[df.columns.get_loc('mode_Minor'):]:
    df[col] = df[col].astype(int)

print(df)

In [ ]:
print(df.head())
print(df.columns)
print(df.dtypes)

<h2>Preliminary Data Exploration</h2>
<p> Here I will be running some simple graphs to give me some visuals regarding the data.</p>


In [ ]:
# Summary Statistics : Distributions
sns.set_style("whitegrid")

# Setting features of interest to a single variable
features_set = ['bpm','danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']

# Defining axes
fig, axes = plt.subplots(nrows=len(features_set), figsize = (15,15))

# Plot the ditribution
for i, feature in enumerate(features_set):
    sns.histplot(df[feature], ax=axes[i], bins=30, kde=True)
    axes[i].set_title(f'Distribution of {feature}', fontsize=11)
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Frequency')
    
# set the spacing between subplots
fig.tight_layout()
plt.show()

In [ ]:
# Finding the top 10 songs based on total streams
streaming_df = df.sort_values(by='streams', ascending = False).head(10) # limiting to the top most 10 results

# Adding artist name to song name
streaming_df['track_artist'] = streaming_df['track_name'] + "-" + streaming_df['artist(s)_name']

# visualizing above data 
print("Streams by Song-Artist:")
# print(streaming_df['track_artist'])

# Plotting the streams with aritst and song information
plt.figure(figsize=(10,5))
sns.barplot(
     x = streaming_df['streams']
    ,y = streaming_df['track_artist']
    ,palette = "flare"
    ,orient = 'h'
     )
plt.title('Top 10 Streams by Song-Artist')
plt.xlabel('Streams in Billions')
plt.ylabel('Track - Artist')
plt.tight_layout()
plt.show()


In [ ]:
# Plotting total streams by artist
streamsxartist_df = df.groupby('artist(s)_name')['streams'].sum().sort_values(ascending=False).head(10) # limiting to the top most 10 results
sns.barplot(
     x = streamsxartist_df.values
    ,y = streamsxartist_df.index
    ,palette = "flare"
    ,orient = 'h'
     )
plt.title('Top 10 Artists with the Highest Streams')
plt.xlabel('Total Streams in Billions')
plt.ylabel('Artist(s) Name')
plt.tight_layout()
plt.show()


<h2>Data Analysis</h2>
<p> Some discovery work has been done to get to know the dataset a little more. Time to run some analysis! Let's try and create a model that can predict how many streams a song gets. </p>

<h3>Correlation</h3>
<p> Before starting any model, I like to look at what features have a high correlation to my dependent variable. This will allow me to understand the relationship between my features and target variable.
For example, I am currently assuming that if a song has high streams then that is synonymous to it being in many Spotify playlists. If it's in Spotify playlists, it is a popular song, hence leading to more streams. Or maybe it's the other way around? Either way, a correlation coefficient will help determine how strong of a relationship these variables have with eachother, if at all. Additionally, here you can find variables that may give you some collinearity issues which would lead to those variables not being in the model.</p> 

In [ ]:
# Cleaning data to remove character values
df_corr = df.drop(['track_name', 'artist(s)_name'], axis = 1)

# Running a simple pearson correlation on the data that has the 'streams' variable set to only numeric values
df_corr.corr('pearson')

<p>As anticipated, the presence of songs in playlists—whether on platforms like Apple Music, Spotify, or Deezer—shows a substantial correlation, roughly around 70%. An intriguing aspect that the available data doesn't clarify is the origin of these playlists. Specifically, it remains uncertain whether these playlists are curated automatically by the platform itself (e.g., Spotify's algorithm) or by individual users.</p>

<p>This distinction holds significance because it raises questions about listener engagement. For instance, when a song achieves chart-topping status or surges in popularity, platforms like Spotify might automatically place it in playlists. Conversely, if a user includes a song in their playlist, it suggests a deliberate choice. This prompts an inquiry: Do songs within user-curated playlists receive more plays compared to those in platform-curated playlists? These are intriguing questions that could offer valuable insights into music listening behavior. However, for the purpose of this analysis and the type of data we are limited to here, we will proceed without addressing them.</p>


<h3>Scaling the Target Variable</h3> 

<p>When examining the data, it becomes apparent that the target variable, 'Streams,' is on a significantly larger scale compared to the feature variables. In such scenarios, it is common practice to consider scaling the variables to ensure uniformity in magnitude.</p>

<p>In the following steps, a logarithmic transformation will be applied to the target variable, and the feature variables will be normalized. It's important to note that while this transformation may impact the ease of interpreting coefficients, it often leads to enhanced performance of machine learning models.</p>


In [ ]:
# Define X : dropping variables that are not of interest at the moment
X = df.drop(['track_name', 'artist(s)_name','streams','released_year','released_month','released_day'], axis = 1)
X_Numeric = (X.loc[:, :'speechiness_%'])

# Define y - using log transformation due to large values
df['streams_log'] = np.log(df['streams'])
y = df.streams_log


<p>Next, an analysis of the relationship between the target variables and the feature variables will be conducted. The following code runs a loop to create graphs for each feature in comparison to the target variable, offering a quick visual overview. In addition, a correlation matrix will be generated for reference.</p>


In [ ]:
# Assuming X is your independent variable and y is your dependent variable

dependent_variable = 'streams_log'

# Get the list of independent variable column names
independent_variables = df.columns

# Setting up subplots

# Calculate the number of independent variables and rows for subplots
num_features = len(independent_variables)

# adds the whole number of rows (calculated in step 1) to either 0 or 1, 
# depending on whether there is an odd number of features left. 
# If there's an odd number of features, it adds an extra row to accommodate 
# the last feature in its own row. If there's an even number of features, it doesn't add an extra row.
num_rows = num_features // 2 + (num_features % 2)

# Create a figure with subplots for multiple scatterplots
fig, axes = plt.subplots(num_rows, 2, figsize=(20, 3*num_rows))

# Iterate through independent variables and create scatterplots
for i, feature in enumerate(independent_variables):
    # Calculate the row and column for the current subplot
    row, col = divmod(i, 2)
    ax = axes[row, col]  # Get the current subplot
    
     # Create a scatterplot of the current independent variable against the dependent variable
    ax.scatter(df[feature], df[dependent_variable])
    
     # Set labels and title for the subplot
    ax.set_xlabel(feature)
    ax.set_ylabel(dependent_variable)
    ax.set_title(f'Scatterplot of {feature} vs. {dependent_variable}')
    
# Remove any empty subplots if the number of features is odd
if num_features % 2 == 1:
    fig.delaxes(axes[num_rows-1, 1])

# Show the scatterplots
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

<h3> Normalizing Features </h3>
<p> My next step is to also standardize my feautures. This will allow alorithms to run faster. Below we will implement Z-score standardization as well as run some visualizations on the effect of the stardization process. </p>

In [ ]:
# Normalizing our features using z score

def zscore_normalize_features(X):
    """
    computes  X, zcore normalized by column
    
    Args:
      X (ndarray (m,n))     : input data, m examples, n features
      
    Returns:
      X_norm (ndarray (m,n)): input normalized by column
      mu (ndarray (n,))     : mean of each feature
      sigma (ndarray (n,))  : standard deviation of each feature
    """
    # find the mean of each column/feature
    mu     = np.mean(X, axis=0)                 # mu will have shape (n,)
    # find the standard deviation of each column/feature
    sigma  = np.std(X, axis=0)                  # sigma will have shape (n,)
    # element-wise, subtract mu for that column from each example, divide by std for that column
    X_norm = (X - mu) / sigma      

    return (X_norm, mu, sigma)

<p> The below code allows you to see the way the data changes before, during, and after normalization. Use the feature_indices to change the options. </p>

In [ ]:
# Specify the feature indices here
feature_indices = [1, 5]  # Change these to the desired feature indices

# Calculate mean and standard deviation
mu = np.mean(X_Numeric, axis=0)
sigma = np.std(X_Numeric, axis=0)

# Calculate X_mean and X_norm
X_mean = (X_Numeric - mu)
X_norm = (X_Numeric - mu) / sigma

# Create scatter plots to visualize the standardization process
fig, ax = plt.subplots(1, 3, figsize=(12, 3))
titles = ["Unnormalized", r"X - $\mu$", "Z-score Normalized"]

for i in range(3):
    if i == 0:
        data_x = X_Numeric.iloc[:, feature_indices[0]]
        data_y = X_Numeric.iloc[:, feature_indices[1]]
    elif i == 1:
        data_x = X_mean.iloc[:, feature_indices[0]]
        data_y = X_mean.iloc[:, feature_indices[1]]
    else:
        data_x = X_norm.iloc[:, feature_indices[0]]
        data_y = X_norm.iloc[:, feature_indices[1]]
    
    ax[i].scatter(data_x, data_y)
    ax[i].set_xlabel(X_Numeric.columns[feature_indices[0]])
    ax[i].set_ylabel(X_Numeric.columns[feature_indices[1]])
    ax[i].set_title(titles[i])
    ax[i].axis('equal')

# Adjust layout and add a main title
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle("Distribution of Features Before, During, After Normalization")

# Show the plot
plt.show()


In [ ]:
# normalize the original features
X_norm, X_mu, X_sigma = zscore_normalize_features(X_Numeric)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma}")
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_Numeric,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm,axis=0)}")

<p> Values after normalization are closer in value than before. Next lets run a linear regression. </p>

<h3> Linear Regression </h3>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
# Defining out training and test data. Setting test data = 20%
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2)

# Set up model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Using statsmodel OLS to find our regression results 
X2 = sm.add_constant(X_norm)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
# setting plot style
plt.style.use('fivethirtyeight')
 
# plotting residual errors in training data
plt.scatter(linear_model.predict(X_train),
            linear_model.predict(X_train) - y_train,
            color="green", s=10,
            label='Train data')
 
# plotting residual errors in test data
plt.scatter(linear_model.predict(X_test),
            linear_model.predict(X_test) - y_test,
            color="blue", s=10,
            label='Test data')
 
# plotting line for zero residual error
plt.hlines(y=0, xmin=0, xmax=50, linewidth=2)
 
# plotting legend
plt.legend(loc='upper right')
 
# plot title
plt.title("Residual errors")
 
# method call for showing the plot
plt.show()


<p> Above you can see the results from the linear regression. Based on the R-Squared value, about 46.7% of the variance in our target variable is being explained by the independent variables. Looking at the F-statistic in combination with the probability, we can conclude that the model is very close to zero which suggests that the model overall is statistically significant.</p>
<p> Overall this model can be improved - one way we can do that is to select more specific features </p>

<h3> Feature Selection </h3>
<p> Here we'll be running a Recursive Feature model to only keep a subset of the most relevant features from our dataset. This should improve the overall model performance, reduce overfitting, and help future algorithms </p>
<p> RFE specifically uses machine learning models to remove the least important features until we are left with our desired number of features.</p>

In [ ]:
from sklearn.metrics import accuracy_score
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFE

In [ ]:
# Defining out training and test data. Setting test data = 20%
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2)

# Set up model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Create an RFE (Recursive Feature Elimination) model and specify the number of features to select
rfe = RFE(estimator=linear_model, n_features_to_select=10)

# Fit the RFE model to your data
rfe.fit(X_train, y_train)

# Get the selected features
selected_features =X_train.columns[rfe.support_]

# Print the selected features
print("Selected Features:")
print(selected_features)

In [ ]:
# Using only selected features
columns_to_keep = selected_features

# Keeping only RFE detected columns
X_norm_rfe = X_norm[columns_to_keep]

# Checking shape
print(f"X_norm Shape:{X_norm_rfe.shape}")


In [ ]:
# Defining out training and test data. Setting test data = 20%
X_train, X_test, y_train, y_test = train_test_split(X_norm_rfe, y, test_size=0.2)

# Set up model
linear_model_rfe = LinearRegression()
linear_model_rfe.fit(X_train, y_train)

# Using statsmodel OLS to find our regression results 
X2 = sm.add_constant(X_norm_rfe)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())


In [ ]:
# setting plot style
plt.style.use('fivethirtyeight')
 
# plotting residual errors in training data
plt.scatter(linear_model_rfe.predict(X_train),
            linear_model_rfe.predict(X_train) - y_train,
            color="green", s=20,
            label='Train data')
 
# plotting residual errors in test data
plt.scatter(linear_model_rfe.predict(X_test),
            linear_model_rfe.predict(X_test) - y_test,
            color="blue", s=20,
            label='Test data')
 
# plotting line for zero residual error
plt.hlines(y=0, xmin=0, xmax=50, linewidth=2)
 
# plotting legend
plt.legend(loc='upper right')
 
# plot title
plt.title("Residual errors")
 
# method call for showing the plot
plt.show()

<p> Investigating the removal of extra columns yielded minimal impact on the model's performance. To enhance the model further, consider generating new variables based on existing fields, such as 'days_from_release_date' derived from the 'release_day' variable. Additionally, you can explore the possibility of identifying top global artists and introducing indicator variables for each of these influential artists. For instance, does a song performed by Taylor Swift serve as a reliable predictor for higher or lower stream counts? While various options exist, we will continue with our current model for this specific analysis, given its significant performance. </p>

<h2> Gradient Descent </h2>
<p> Next we can try a different kind of model which is called Gradient Descent. The overall purpose of this model is to optimize a function. It minimizes the cost function while at the same time adjusting the model's parameters bit by bit. For this example I will continue to use the normalized target and feature variables, but I will go back to using all the features that were included prior to utilizing RFE. </p>

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
# Define X : dropping variables that are not of interest at the moment
X = df.drop(['track_name', 'artist(s)_name','streams','released_year','released_month','released_day'], axis = 1)
X_Numeric = (X.loc[:, :'speechiness_%'])

# Define y - using log transformation due to large values
df['streams_log'] = np.log(df['streams'])
y = df.streams_log



In [ ]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_Numeric)

# Defining out training and test data. Setting test data = 20%
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2)

print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_train,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_Numeric,axis=0)}")

In [ ]:
# create and fit regression model
sgdr = SGDRegressor(max_iter=1000)
sgdr.fit(X_train, y_train)
print(sgdr)
print(f"number of iterations completed: {sgdr.n_iter_}, number of weight updates: {sgdr.t_}")


<p> The above results state that the model went through 17 iterations and updated the parameters 11085 times </p>

In [ ]:
# looking at parameters from model using normalized values
b_norm = sgdr.intercept_
w_norm = sgdr.coef_
print(f"model parameters:                   w: {w_norm}, b:{b_norm}")

In [ ]:
# make a prediction using sgdr.predict()
y_pred_sgd = sgdr.predict(X_test)

# make a prediction using w,b. 
y_pred = np.dot(X_test, w_norm) + b_norm  
print(f"prediction using np.dot() and sgdr.predict match: {(y_pred == y_pred_sgd).all()}")

print(f"Prediction on training set:\n{y_pred[:4]}" )
print(f"Target values \n{y_train[:4]}")

In [ ]:
# plot predictions and targets vs original features    
fig, ax = plt.subplots(1, 6, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:, i], y_train, label='target')
    ax[i].set_xlabel(f'Feature {i}')
    ax[i].scatter(X_test[:, i], y_pred, color="orange", label='predict')
ax[0].set_ylabel("Price")
ax[0].legend()
fig.suptitle("Target versus prediction using z-score normalized model")
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # RMSE
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)

<p> So these are the results of our gradient descent model. Let's break them down! </p>
<p> <b>MSE</b> : measures the average squared difference between the predicted values and the actual target values. A lower MSE = better predictions. My current model has quite a bit of error when compared to the actual targets </p>
<p> <b>RMSE</b> : is simply the square root of the MSE  - very similar interpretation to the above example. </p>
<p> <b>MAE</b> : This measures the absolute difference between the predicted values and the actual target values. We also want a lower value here. </p>
<p><b>R2</b> : This measures how well the model explains the variance in the target variables. This model explains about 50% of the variability in the target variable. </p>

<h2>Concluding Thoughts</h2>
<p> In summary, the predictive capability of this model can be described as average at best. However, there is ample room for enhancement, which I intend to explore gradually. This endeavor will involve the thoughtful incorporation of additional high-quality variables into the dataset and addressing outliers, a few of which are present in the data, with the aim of enhancing the model's effectiveness. </p>
<p> The primary aim of this analysis was twofold: to gain exposure to this dataset and to apply a diverse array of data analysis techniques. Throughout this exploration, I leveraged various functions to achieve comparable results. This approach reflects my curiosity and serves as a means to assess the variability of results across different tools. It also plays a pivotal role in gauging my progress and comprehension of the subject matter.</p>

<h4>Feedback</h4>
<p> I love learning, but with learning you have success and you most definetely have failures. Although I'm pretty well aquainted with the tools used in this analysis, if you find an error or have a suggestion, please do not hesistate to reach out! You can find my information on my webpage. Thank you! <p>

<p>Data Sourced from Kaggle : <a>https://www.kaggle.com/datasets/nelgiriyewithana/top-spotify-songs-2023/data</a><p>